In [656]:
import pandas as pd
from pathlib import Path
import numpy as np

In [658]:
# -----------------------------
# CONFIGURATION
# -----------------------------
DATA_DIR = Path("../out/equity/history")   # change to your folder path
DATE_COL = "datetime"          # or "date"
PRICE_COL = "close"            # or "price"
ISIN_COL = "isin"
COMPANY_COL = "company"   # change to "company" if needed
MDM_FILE = "../data/normalized/mdm.xlsx"

In [660]:
# -----------------------------
# READ ALL FILES
# -----------------------------
all_dfs = []

for file in DATA_DIR.glob("*.xlsx"):
    try:
        df = pd.read_excel(file, engine="openpyxl")
    except Exception as e:
        print(f"Skipping {file.name}: {e}")
        continue

   # normalize column names
    df.columns = df.columns.str.lower().str.strip()

    # parse datetime
    df[DATE_COL] = pd.to_datetime(df[DATE_COL])

    # year / month
    df["year"] = df[DATE_COL].dt.year
    df["month"] = df[DATE_COL].dt.month
    df["month_start"] = df[DATE_COL].dt.to_period("M").dt.to_timestamp()


    # keep only required columns (safe & clean)
    df = df[[ISIN_COL, COMPANY_COL, "month_start", "year", "month", PRICE_COL]]

    all_dfs.append(df)
    

Skipping ~$INE009A01021-Infosys Ltd.xlsx: File is not a zip file


In [661]:
# -----------------------------
# MASTER DATAFRAME
# -----------------------------
master_df = pd.concat(all_dfs, ignore_index=True)

print("Master dataframe shape:", master_df.shape)
print(master_df.head())

Master dataframe shape: (32739, 6)
           isin             company month_start  year  month    close
0  INE018A01030  Larsen & Tubro Ltd  2014-10-01  2014     10  1050.00
1  INE018A01030  Larsen & Tubro Ltd  2014-10-01  2014     10  1060.73
2  INE018A01030  Larsen & Tubro Ltd  2014-10-01  2014     10  1102.23
3  INE018A01030  Larsen & Tubro Ltd  2014-11-01  2014     11  1104.00
4  INE018A01030  Larsen & Tubro Ltd  2014-11-01  2014     11  1116.00


In [666]:
# -----------------------------
# MONTHLY AVERAGE
# -----------------------------
df_eq_mth = (
    master_df
    .groupby(
        [ISIN_COL, COMPANY_COL, "year", "month", "month_start"],
        as_index=False
    )[PRICE_COL]
    .mean()
)

# rounding
df_eq_mth[PRICE_COL] = df_eq_mth[PRICE_COL].round(2)
# rename column
df_eq_mth.rename(columns={PRICE_COL: "avg_close"}, inplace=True)

# optional sorting
df_eq_mth.sort_values(
    [ISIN_COL, "year", "month"],
    inplace=True
)

print("Monthly aggregation:")
print(monthly_avg.head())

Monthly aggregation:
           isin      company  year  month month_start       close
0  INE009A01021  Infosys Ltd  2015      6  2015-06-01  499.476364
1  INE009A01021  Infosys Ltd  2015      7  2015-07-01  509.363043
2  INE009A01021  Infosys Ltd  2015      8  2015-08-01  556.275238
3  INE009A01021  Infosys Ltd  2015      9  2015-09-01  549.896500
4  INE009A01021  Infosys Ltd  2015     10  2015-10-01  626.651000


In [567]:
OUTPUT_DIR = Path("../out/equity/aggregated")
OUTPUT_DIR.mkdir(exist_ok=True)

master_df.to_csv(OUTPUT_DIR / "equity_master.csv", index=False)
df_eq_mth.to_csv(OUTPUT_DIR / "equity_monthly_avg.csv", index=False)


In [568]:
def clean_isin(series: pd.Series) -> pd.Series:
    return (
        series
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", "", regex=True)
        .str.upper()
    )

In [569]:
# Read sheets
df_equity = pd.read_excel(MDM_FILE, sheet_name="Equity", engine="openpyxl")
df_txn = pd.read_excel(MDM_FILE, sheet_name="Equity Transaction", engine="openpyxl")

# Normalize columns
df_equity.columns = df_equity.columns.str.lower().str.strip()
df_txn.columns = df_txn.columns.str.lower().str.strip()

df_equity["isin"] = clean_isin(df_equity["isin"])
df_txn["isin"] = clean_isin(df_txn["isin"])

df_txn["trade date"] = pd.to_datetime(df_txn["trade date"])
df_txn["year"] = df_txn["trade date"].dt.year
df_txn["month"] = df_txn["trade date"].dt.month
df_txn["month_start"] = df_txn["trade date"].dt.to_period("M").dt.to_timestamp()

In [570]:
df_equity.head()

,isin,company,nse symbol,bse code,incorporation date,isin creation date,icici breeze code
0,INE238A01034,Axis Bank Ltd,AXISBANK,532215,03-Dec-1993,NaT,AXIBAN
1,INE457A01014,Bank of Maharashtra,MAHABANK,532525,16-Sep-1935,NaT,BANMAH
2,INE323A01026,Bosch Ltd,BOSCHLTD,500530,12-Nov-1951,NaT,BOSLIM
3,INE925R01014,CMS Info Systems Ltd,CMSINFO,543441,26-Mar-2008,NaT,CMSIN
4,INE335K01011,Coffee Day Enterprises Ltd,COFFEEDAY,539436,20-Jun-2008,NaT,COFDAY


In [571]:
df_txn.head()

,person,account,trade date,isin,security,segment,action,quantity,price/share,net amount,brokerageperunit,netbrokerage,stt,stamp duty,gst,total amount,year,month,month_start
0,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,1.25,25.0,7.00,0.45,4.52,7111.97,2020,4,2020-04-01
1,Arun Venkatesan,ICICI,2020-07-06,INE040A01034,HDFC BANK LTD,CASH,Buy,10,1100.00,11000.0,6.05,60.5,11.00,2.00,10.95,11084.45,2020,7,2020-07-01
2,Arun Venkatesan,ICICI,2021-12-20,INE562A01011,INDIAN BANK,CASH,Buy,62,136.45,8459.9,0.75,46.5,0.00,0.00,0.00,8506.40,2021,12,2021-12-01
3,Arun Venkatesan,ICICI,2021-12-20,INE562A01011,INDIAN BANK,CASH,Buy,38,136.45,5185.1,0.75,28.5,13.99,2.00,13.55,5243.14,2021,12,2021-12-01
4,Arun Venkatesan,ICICI,2021-12-23,INE040A01034,HDFC BANK LTD,CASH,Buy,10,1450.00,14500.0,7.97,79.7,14.40,2.18,14.40,14610.68,2021,12,2021-12-01


In [572]:
df_eq = (
    df_txn
    .merge(
        df_equity[
            ["isin", "company", "nse symbol", "bse code"]
        ],
        on="isin",
        how="left"
    )
)


In [573]:
#df_eq.head()
df_infy = df_eq[df_eq["isin"].str.contains("INE323A01026", case=False)]
df_infy.head(50)

,person,account,trade date,isin,security,segment,action,quantity,price/share,net amount,...,stt,stamp duty,gst,total amount,year,month,month_start,company,nse symbol,bse code
44,Arun Venkatesan,IIFL,2020-03-27,INE323A01026,BOSCHLTD,CASH,Buy,6,10000.0,60000.0,...,NaN,NaN,NaN,60300.00,2020,3,2020-03-01,Bosch Ltd,BOSCHLTD,500530.0
96,Kurinji Malar Paranthaman,GEOJIT,2020-03-27,INE323A01026,BOSCH LIMITED,CASH,Buy,2,10050.0,20100.0,...,20.0,1.21,10.98,20192.49,2020,3,2020-03-01,Bosch Ltd,BOSCHLTD,500530.0


In [574]:
df_joined = (
    df_eq
    .merge(
        df_eq_mth,
        on="isin",
        how="left",
        suffixes=("", "_mth")
    )
)

# apply year & month filter
df_joined = df_joined[
    (df_joined["month_start_mth"] >= df_joined["month_start"])
]


In [575]:
df_joined

,person,account,trade date,isin,security,segment,action,quantity,price/share,net amount,...,month,month_start,company,nse symbol,bse code,company_mth,year_mth,month_mth,month_start_mth,avg_close
0,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,...,4,2020-04-01,ICICI Bank Ltd,ICICIBANK,532174.0,ICICI Bank Ltd,2020.0,4.0,2020-04-01,375.30
1,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,...,4,2020-04-01,ICICI Bank Ltd,ICICIBANK,532174.0,ICICI Bank Ltd,2020.0,5.0,2020-05-01,320.24
2,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,...,4,2020-04-01,ICICI Bank Ltd,ICICIBANK,532174.0,ICICI Bank Ltd,2020.0,6.0,2020-06-01,350.66
3,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,...,4,2020-04-01,ICICI Bank Ltd,ICICIBANK,532174.0,ICICI Bank Ltd,2020.0,7.0,2020-07-01,361.70
4,Arun Venkatesan,ICICI,2020-04-28,INE090A01021,ICICI BANK LTD.,CASH,Buy,20,353.75,7075.0,...,4,2020-04-01,ICICI Bank Ltd,ICICIBANK,532174.0,ICICI Bank Ltd,2020.0,8.0,2020-08-01,370.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6692,Kurinji Malar Paranthaman,GEOJIT,2025-01-14,INE379A01028,ITC Hotels Limited,SPLIT,Split,5,0.00,0.0,...,1,2025-01-01,ITC Hotels Ltd,ITCHOTELS,544325.0,ITC Hotels Ltd,2025.0,9.0,2025-09-01,240.02
6693,Kurinji Malar Paranthaman,GEOJIT,2025-01-14,INE379A01028,ITC Hotels Limited,SPLIT,Split,5,0.00,0.0,...,1,2025-01-01,ITC Hotels Ltd,ITCHOTELS,544325.0,ITC Hotels Ltd,2025.0,10.0,2025-10-01,217.80
6694,Kurinji Malar Paranthaman,GEOJIT,2025-01-14,INE379A01028,ITC Hotels Limited,SPLIT,Split,5,0.00,0.0,...,1,2025-01-01,ITC Hotels Ltd,ITCHOTELS,544325.0,ITC Hotels Ltd,2025.0,11.0,2025-11-01,209.56
6695,Kurinji Malar Paranthaman,GEOJIT,2025-01-14,INE379A01028,ITC Hotels Limited,SPLIT,Split,5,0.00,0.0,...,1,2025-01-01,ITC Hotels Ltd,ITCHOTELS,544325.0,ITC Hotels Ltd,2025.0,12.0,2025-12-01,197.74


In [581]:
# df_joined
df_infy = df_joined[df_joined["nse symbol"].str.contains("MAHABANK", case=False)]
df_infy.head(50)

,person,account,trade date,isin,security,segment,action,quantity,price/share,net amount,...,month,month_start,company,nse symbol,bse code,company_mth,year_mth,month_mth,month_start_mth,avg_close
4843,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2024.0,12.0,2024-12-01,53.06
4844,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,1.0,2025-01-01,50.86
4845,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,2.0,2025-02-01,49.23
4846,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,3.0,2025-03-01,46.69
4847,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,4.0,2025-04-01,48.13
4848,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,5.0,2025-05-01,51.00
4849,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,6.0,2025-06-01,55.14
4850,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,7.0,2025-07-01,56.87
4851,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,8.0,2025-08-01,54.53
4852,Arun Venkatesan,IIFL,2021-06-09,INE457A01014,MAHABANK,CASH,Buy,500,26.85,13425.0,...,6,2021-06-01,Bank of Maharashtra,MAHABANK,532525.0,Bank of Maharashtra,2025.0,9.0,2025-09-01,54.68


In [624]:
# Calculate Metrics
df_joined["invested_amt"] = np.where(
    df_joined["net amount"].isna() | (df_joined["net amount"] == 0),
    df_joined["quantity"] * df_joined["price/share"],
    df_joined["net amount"]
)

df_joined["current_amt"] = (
    df_joined["quantity"] * df_joined["avg_close"]
)

# rounding (financial safe)
df_joined["invested_amt"] = df_joined["invested_amt"].round(2)
df_joined["current_amt"] = df_joined["current_amt"].round(2)


# 10% Valuation calculation
ANNUAL_RATE = 0.10
MONTHLY_RATE = (1 + ANNUAL_RATE) ** (1/12) - 1

df_joined["trade date"] = pd.to_datetime(df_joined["trade date"])
df_joined["month_start"] = pd.to_datetime(df_joined["month_start"])
df_joined["invested_amt"] = pd.to_numeric(df_joined["invested_amt"], errors="coerce")

df_joined["months_elapsed"] = (
    (df_joined["month_start_mth"].dt.year - df_joined["trade date"].dt.year) * 12 +
    (df_joined["month_start_mth"].dt.month - df_joined["trade date"].dt.month)
)

# keep only valid forward-looking months
df_joined = df_joined[df_joined["months_elapsed"] >= 0]

# Calculate compounded valuation
df_joined["compound_value_10pct"] = (
    df_joined["invested_amt"] *
    (1 + MONTHLY_RATE) ** df_joined["months_elapsed"]
)

df_joined["compound_value_10pct"] = df_joined["compound_value_10pct"].round(2)

df_joined["compound_pnl_10pct"] = (
    df_joined["current_amt"] - df_joined["compound_value_10pct"]
).round(2)

# Handle BUY / SELL correctly
sign = np.where(df_joined["action"].str.upper() == "SELL", -1, 1)

df_joined["compound_value_10pct"] *= sign
df_joined["compound_pnl_10pct"] *= sign

df_joined.loc[
    df_joined["action"].str.upper().isin(["BONUS", "DIVIDEND"]),
    ["compound_value_10pct", "compound_pnl_10pct"]
] = 0

#Calculate CAGR
df_joined["years_elapsed"] = (
    df_joined["months_elapsed"] / 12
)


df_joined["cagr"] = (
    (df_joined["current_amt"] / df_joined["invested_amt"])
    ** (1 / df_joined["years_elapsed"])
    - 1
)

df_joined.loc[
    (df_joined["years_elapsed"] <= 0) |
    (df_joined["invested_amt"] <= 0),
    "cagr"
] = 0

df_joined["cagr_pct"] = (df_joined["cagr"] * 100).round(2)


In [626]:
final_df = df_joined[
    [
        "person",
        "account",
        "isin",
        "company",
        "nse symbol",
        "trade date",
        "segment",
        "action",
        "quantity",
        "price/share",
        "net amount",
        "year_mth",
        "month_mth",
        "avg_close",
        "invested_amt",
        "current_amt",
        "months_elapsed",
        "month_start",
        "compound_value_10pct",
        "compound_pnl_10pct",
        "cagr_pct"
    ]
].sort_values(
    ["isin", "year_mth", "month_mth"]
)


In [628]:
final_df

,person,account,isin,company,nse symbol,trade date,segment,action,quantity,price/share,...,year_mth,month_mth,avg_close,invested_amt,current_amt,months_elapsed,month_start,compound_value_10pct,compound_pnl_10pct,cagr_pct
2535,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2015-06-15,CASH,BUY,3,992.00,...,2015.0,6.0,499.48,2976.00,1498.44,0,2015-06-01,2976.00,-1477.56,0.00
2663,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2015-06-15,CASH,BUY,10,992.00,...,2015.0,6.0,499.48,9920.00,4994.80,0,2015-06-01,9920.00,-4925.20,0.00
2791,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2015-06-15,CASH,BUY,20,990.00,...,2015.0,6.0,499.48,19800.00,9989.60,0,2015-06-01,19800.00,-9810.40,0.00
2919,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2015-06-15,CASH,BUY,20,992.00,...,2015.0,6.0,499.48,19840.00,9989.60,0,2015-06-01,19840.00,-9850.40,0.00
2536,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2015-06-15,CASH,BUY,3,992.00,...,2015.0,7.0,509.36,2976.00,1528.08,1,2015-06-01,2999.73,-1471.65,-99.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,Arun Venkatesan,ICICI,INE848E01016,NHPC Ltd,NHPC,2022-11-15,CASH,Sell,1,43.75,...,2025.0,11.0,80.41,43.75,80.41,36,2022-11-01,-58.23,-22.18,22.49
552,Arun Venkatesan,ICICI,INE848E01016,NHPC Ltd,NHPC,2022-11-15,CASH,Buy,1,43.80,...,2025.0,12.0,77.09,43.80,77.09,37,2022-11-01,58.76,18.33,20.12
591,Arun Venkatesan,ICICI,INE848E01016,NHPC Ltd,NHPC,2022-11-15,CASH,Sell,1,43.75,...,2025.0,12.0,77.09,43.75,77.09,37,2022-11-01,-58.70,-18.39,20.17
553,Arun Venkatesan,ICICI,INE848E01016,NHPC Ltd,NHPC,2022-11-15,CASH,Buy,1,43.80,...,2026.0,1.0,80.64,43.80,80.64,38,2022-11-01,59.23,21.41,21.26


In [596]:
# filter for INFY
df_infy = final_df[final_df["nse symbol"].str.contains("INFY", case=False)]

# get latest trade date
latest_date = df_infy["trade date"].max()

# row(s) for the latest trade
latest_infy = df_infy[df_infy["trade date"] == latest_date]

latest_infy.head(50)


,person,account,isin,company,nse symbol,trade date,segment,action,quantity,price/share,net amount,year_mth,month_mth,avg_close,invested_amt,current_amt,months_elapsed,month_start,compound_value_10pct,compound_pnl_10pct
5561,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2024.0,12.0,1933.29,7884.0,7733.16,0,2024-12-01,7884.00,-150.84
5562,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,1.0,1893.27,7884.0,7573.08,1,2024-12-01,7946.87,-373.79
5563,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,2.0,1838.88,7884.0,7355.52,2,2024-12-01,8010.24,-654.72
5564,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,3.0,1633.31,7884.0,6533.24,3,2024-12-01,8074.11,-1540.87
5565,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,4.0,1458.01,7884.0,5832.04,4,2024-12-01,8138.50,-2306.46
5566,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,5.0,1557.95,7884.0,6231.80,5,2024-12-01,8203.39,-1971.59
5567,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,6.0,1596.23,7884.0,6384.92,6,2024-12-01,8268.81,-1883.89
5568,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,7.0,1581.56,7884.0,6326.24,7,2024-12-01,8334.75,-2008.51
5569,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,8.0,1464.26,7884.0,5857.04,8,2024-12-01,8401.21,-2544.17
5570,Arun Venkatesan,IIFL,INE009A01021,Infosys Ltd,INFY,2024-12-18,CASH,Buy,4,1971.0,7884.0,2025.0,9.0,1491.90,7884.0,5967.60,9,2024-12-01,8468.20,-2500.60


In [598]:
OUTPUT_DIR = Path("../out/equity/aggregated")
OUTPUT_DIR.mkdir(exist_ok=True)

final_df.to_excel(OUTPUT_DIR / "equity_monthly_agg.xlsx", index=False)

In [650]:
cagr_isin = (
    df_joined
    .groupby(["isin", "company", "year_mth", "month_mth"], as_index=False)
    .agg(
        total_invested=("invested_amt", "sum"),
        current_value=("current_amt", "sum"),
        max_years=("years_elapsed", "max")
    )
)

# calculate CAGR safely
cagr_isin["cagr"] = (
    (cagr_isin["current_value"] / cagr_isin["total_invested"])
    ** (1 / cagr_isin["max_years"])
    - 1
)

# handle invalid cases
cagr_isin.loc[
    (cagr_isin["max_years"] <= 0) |
    (cagr_isin["total_invested"] <= 0),
    "cagr"
] = 0

cagr_isin["cagr_pct"] = (cagr_isin["cagr"] * 100).round(2)



In [652]:
cagr_isin

,isin,company,year_mth,month_mth,total_invested,current_value,max_years,cagr,cagr_pct
0,INE009A01021,Infosys Ltd,2015.0,6.0,52536.00,26472.44,0.000000,0.000000,0.00
1,INE009A01021,Infosys Ltd,2015.0,7.0,52536.00,26996.08,0.083333,-0.999661,-99.97
2,INE009A01021,Infosys Ltd,2015.0,8.0,52536.00,29482.84,0.166667,-0.968763,-96.88
3,INE009A01021,Infosys Ltd,2015.0,9.0,52536.00,29144.70,0.250000,-0.905287,-90.53
4,INE009A01021,Infosys Ltd,2015.0,10.0,52536.00,33212.45,0.333333,-0.747343,-74.73
...,...,...,...,...,...,...,...,...,...
1351,INE848E01016,NHPC Ltd,2025.0,9.0,87.55,166.92,2.833333,0.255777,25.58
1352,INE848E01016,NHPC Ltd,2025.0,10.0,87.55,172.30,2.916667,0.261276,26.13
1353,INE848E01016,NHPC Ltd,2025.0,11.0,87.55,160.82,3.000000,0.224695,22.47
1354,INE848E01016,NHPC Ltd,2025.0,12.0,87.55,154.18,3.083333,0.201461,20.15


In [654]:
OUTPUT_DIR = Path("../out/equity/aggregated")
OUTPUT_DIR.mkdir(exist_ok=True)

cagr_isin.to_excel(OUTPUT_DIR / "equity_monthly_cagr.xlsx", index=False)

In [668]:
# Calculate XIRR

# Create cash-flow column
df_cf = df_joined.copy()

df_cf["cash_flow"] = 0.0

# BUY → negative cash flow
df_cf.loc[
    df_cf["action"].str.upper() == "BUY",
    "cash_flow"
] = -df_cf["invested_amt"]

# SELL → positive cash flow
df_cf.loc[
    df_cf["action"].str.upper() == "SELL",
    "cash_flow"
] = df_cf["net amount"]

# Ignore bonus/dividend
df_cf = df_cf[
    ~df_cf["action"].str.upper().isin(["BONUS"])
]


In [670]:
# Add final valuation cash flow

valuation_date = df_cf["month_start"].max()

final_valuation = (
    df_cf
    .groupby(["isin", "company"], as_index=False)
    .agg(final_value=("current_amt", "sum"))
)

final_valuation["trade date"] = valuation_date
final_valuation["cash_flow"] = final_valuation["final_value"]

final_valuation = final_valuation[
    ["isin", "company", "trade date", "cash_flow"]
]

# Append to cash-flow table
df_cf = pd.concat(
    [
        df_cf[["isin", "company", "trade date", "cash_flow"]],
        final_valuation
    ],
    ignore_index=True
)


In [672]:
# XIRR function (Newton–Raphson implementation)

def xirr(cash_flows, dates, guess=0.1):
    """
    cash_flows: list/array of floats
    dates: list/array of datetime
    """
    cash_flows = np.array(cash_flows, dtype=float)
    dates = pd.to_datetime(dates)

    # convert dates to year fractions
    t0 = dates.min()
    years = (dates - t0).dt.days / 365.25

    def npv(rate):
        return np.sum(cash_flows / (1 + rate) ** years)

    def d_npv(rate):
        return np.sum(-years * cash_flows / (1 + rate) ** (years + 1))

    rate = guess
    for _ in range(100):
        value = npv(rate)
        deriv = d_npv(rate)
        if deriv == 0:
            break
        new_rate = rate - value / deriv
        if abs(new_rate - rate) < 1e-6:
            return new_rate
        rate = new_rate

    return np.nan



In [688]:
# Calculate XIRR per ISIN

xirr_isin = []

for isin, g in df_cf.groupby("isin"):
    if g["cash_flow"].sum() <= 0:
        continue  # XIRR undefined

    rate = xirr(
        cash_flows=g["cash_flow"],
        dates=g["trade date"]
    )

    xirr_isin.append({
        "isin": isin,
        "company": g["company"].iloc[0],
        "xirr": rate,
        "xirr_pct": round(rate * 100, 2) if pd.notna(rate) else None
    })

xirr_isin = pd.DataFrame(xirr_isin)


In [690]:
xirr_isin.to_excel(OUTPUT_DIR / "equity_monthly_xirr.xlsx", index=False)

In [676]:
print(xirr_isin)

            isin                                    company       xirr  \
0   INE009A01021                                Infosys Ltd   0.010608   
1   INE018A01030                         Larsen & Tubro Ltd   0.080361   
2   INE040A01034                              HDFC Bank Ltd   0.020774   
3   INE040H01021                         Suzlon Engergy Ltd   1.510713   
4   INE053A01029                  Indian Hotels Company Ltd   0.351852   
5   INE081A01020                             Tata Steel Ltd        NaN   
6   INE090A01021                             ICICI Bank Ltd   0.142265   
7   INE092T01019                        IDFC First Bank Ltd   0.093798   
8   INE114A01011        Steel Authority of India Ltd (SAIL)   0.032136   
9   INE155A01022  Tata Motors Passenger Vehicles Ltd (TMPV)   0.004387   
10  INE1NPP01017                   Siemens Energy India Ltd        NaN   
11  INE1TAE01010                            Tata Motors Ltd        NaN   
12  INE238A01034                      

In [678]:
# Prepare Portfolio Cash Flows

df_port_cf = df_joined.copy()

df_port_cf["cash_flow"] = 0.0

# BUY → negative
df_port_cf.loc[
    df_port_cf["action"].str.upper() == "BUY",
    "cash_flow"
] = -df_port_cf["invested_amt"]

# SELL → positive
df_port_cf.loc[
    df_port_cf["action"].str.upper() == "SELL",
    "cash_flow"
] = df_port_cf["net amount"]

# ignore bonus / non-cash actions
df_port_cf = df_port_cf[
    ~df_port_cf["action"].str.upper().isin(["BONUS"])
]

df_port_cf = df_port_cf[
    ["trade date", "cash_flow"]
]


In [686]:
# Add Final Portfolio Valuation (CRITICAL)
valuation_date = df_joined["month_start"].max()

final_portfolio_value = df_joined["current_amt"].sum()

final_row = pd.DataFrame(
    {
        "trade date": [valuation_date],
        "cash_flow": [final_portfolio_value]
    }
)

df_port_cf = pd.concat(
    [df_port_cf, final_row],
    ignore_index=True
)


In [684]:
# Calculate Portfolio-Level XIRR

portfolio_xirr = xirr(
    cash_flows=df_port_cf["cash_flow"],
    dates=df_port_cf["trade date"]
)

portfolio_xirr_pct = round(portfolio_xirr * 100, 2)

print(f"Portfolio XIRR: {portfolio_xirr_pct}%")


Portfolio XIRR: 20.62%
